# Bibliotecas

In [494]:
# ! pip install selenium --quiet
# ! apt-get update --quiet
# ! apt install chromium-chromedriver --quiet

In [495]:
# from    selenium import webdriver
import  pandas as pd
# from    selenium import webdriver
import  numpy as np
import  re
import  plotly.express as px
from    plotly.subplots import make_subplots
import  plotly.graph_objects as go
from    sklearn.decomposition import PCA
from    sklearn.preprocessing import OneHotEncoder

In [496]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Webdriver

In [497]:
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# Nikola Jokic

In [498]:
# wd = webdriver.Chrome(options=options)

# temporadas_jokic = np.arange(2016, 2024)

# lista_dfs = []

# # Coleta os dados do jogador de 2016 a 2023
# for ano in temporadas_jokic:
#   site = 'https://www.espn.com.br/nba/jogador/jogo-a-jogo/_/id/3112335/tipo/nba/ano/' + str(ano)

#   wd.get(site)

#   html = wd.page_source

#   scrapping = pd.read_html(html)

#   df_stats_ano = pd.concat([scrapping[0],
#                             scrapping[1],
#                             scrapping[2],
#                             scrapping[3],
#                             scrapping[4],
#                             scrapping[5],
#                             scrapping[6]])

#   df_stats_ano.insert(0, 'Temporada', ano)

#   lista_dfs.append(df_stats_ano)

#   df_stats_jokic = pd.concat(lista_dfs).reset_index(drop=True)

In [499]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset raw from csv

In [500]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
                             encoding='utf-8',
                             sep=',')

In [501]:
df_stats_jokic.head()

,Temporada,Date,ADV,Resultado,MIN,FG,FG%,3PT,3P%,FT,FT%,REB,V,BLK,STL,PF,TO,PTS,Estat. Temporada Regular
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,NaN
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0,NaN
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,NaN
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,NaN
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,NaN


# Data wrangling

In [502]:
df_stats_jokic.isnull().sum()

Temporada                     0
Date                          2
ADV                           2
Resultado                     2
MIN                           0
FG                            0
FG%                           0
3PT                           0
3P%                           0
FT                            0
FT%                           0
REB                           0
V                             0
BLK                           0
STL                           0
PF                            0
TO                            0
PTS                           0
Estat. Temporada Regular    654
dtype: int64

In [503]:
df_stats_jokic.drop(columns='Estat. Temporada Regular', inplace=True)

df_stats_jokic.dropna(axis=0, how='any', inplace=True)

In [504]:
df_stats_jokic.isnull().sum()

Temporada    0
Date         0
ADV          0
Resultado    0
MIN          0
FG           0
FG%          0
3PT          0
3P%          0
FT           0
FT%          0
REB          0
V            0
BLK          0
STL          0
PF           0
TO           0
PTS          0
dtype: int64

In [505]:
renamed_columns = {'Date': 'Data',
                   'V': 'Assistencias',
                   'PF': 'Faltas',
                   'TO': 'Tunovers',
                   'MIN': 'Minutos',
                   'REB': 'Rebotes',
                   'BLK': 'Bloqueios',
                   'STL': 'Desarmes',
                   'PTS': 'Pontos'}

df_stats_jokic.rename(columns=renamed_columns, inplace=True)

df_stats_jokic.head()

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0


In [506]:
# Remoção de linhas com texto de nome de meses
df_stats_jokic = df_stats_jokic[(df_stats_jokic['Data']   != 'january')
                                & (df_stats_jokic['Data'] != 'february')
                                & (df_stats_jokic['Data'] != 'march')
                                & (df_stats_jokic['Data'] != 'april')
                                & (df_stats_jokic['Data'] != 'may')
                                & (df_stats_jokic['Data'] != 'june')
                                & (df_stats_jokic['Data'] != 'july')
                                & (df_stats_jokic['Data'] != 'august')
                                & (df_stats_jokic['Data'] != 'september')
                                & (df_stats_jokic['Data'] != 'october')
                                & (df_stats_jokic['Data'] != 'november')
                                & (df_stats_jokic['Data'] != 'december')]

In [507]:
# Verifica a remoção de linhas com texto de nome de meses
df_stats_jokic[(df_stats_jokic['Data']   == 'january')
               | (df_stats_jokic['Data'] == 'february')
               | (df_stats_jokic['Data'] == 'march')
               | (df_stats_jokic['Data'] == 'april')
               | (df_stats_jokic['Data'] == 'may')
               | (df_stats_jokic['Data'] == 'june')
               | (df_stats_jokic['Data'] == 'july')
               | (df_stats_jokic['Data'] == 'august')
               | (df_stats_jokic['Data'] == 'september')
               | (df_stats_jokic['Data'] == 'october')
               | (df_stats_jokic['Data'] == 'november')
               | (df_stats_jokic['Data'] == 'december')]

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos


In [508]:
# Regex de informações das colunas para novas colunas exclusivas
df_stats_jokic['FG_Tentativas'] = df_stats_jokic['FG'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['FG_Acertos'] = df_stats_jokic['FG'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['3PT_Tentativas'] = df_stats_jokic['3PT'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['3PT_Acertos'] = df_stats_jokic['3PT'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['FT_Tentativas'] = df_stats_jokic['FT'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['FT_Acertos'] = df_stats_jokic['FT'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['Adversario'] = df_stats_jokic['ADV'].str.extract(r'(?:em|x)([A-Z]+)')  # (?:em|x) - non-capturing group that matches 'em' or 'x'

df_stats_jokic['Placar'] = df_stats_jokic['Resultado'].str.extract(r'(\d+-\d+)')

df_stats_jokic['V_D'] = df_stats_jokic['Resultado'].str.extract(r'(^D|V)')

df_stats_jokic['Casa_Fora'] = df_stats_jokic['ADV'].str.extract(r'(em|x)')

df_stats_jokic['Dia_semana'] = df_stats_jokic['Data'].str.extract(r'([A-Za-z0-9_À-ÿ]+)')

df_stats_jokic['Mes'] = df_stats_jokic['Data'].str.extract(r'\b\d+/\b(\d+)')

In [509]:
df_stats_jokic['Dia_semana'] = (df_stats_jokic['Dia_semana'].map({'Seg': 0,
                                                                  'Ter': 1,
                                                                  'Qua': 2,
                                                                  'Qui': 3,
                                                                  'Sex': 4,
                                                                  'Sáb': 5,
                                                                  'Dom': 6}))  # Dias da semana no padrão python

In [510]:
pd.set_option('display.max_columns', None)

df_stats_jokic['%FG'] = round(df_stats_jokic['FG_Acertos'] / df_stats_jokic['FG_Tentativas'], 2)

df_stats_jokic['%3PT'] = round(df_stats_jokic['3PT_Acertos'] / df_stats_jokic['3PT_Tentativas'], 2)

df_stats_jokic['%FT'] = round(df_stats_jokic['FT_Acertos'] / df_stats_jokic['FT_Tentativas'], 2)

df_stats_jokic['Casa_Fora'] = df_stats_jokic['Casa_Fora'].map({'em': 'fora','x': 'casa'})

In [511]:
df_stats_jokic.head()

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Placar,V_D,Casa_Fora,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,POR,107-99,D,fora,3,4,0.70,0.50,1.00
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,UTAH,100-84,D,casa,6,4,0.78,0.00,0.83
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,SA,102-98,V,casa,5,4,0.33,0.00,1.00
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,OKC,124-102,D,casa,2,4,0.60,0.33,NaN
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,SAC,115-106,D,casa,6,4,0.33,0.00,1.00


In [512]:
df_stats_jokic.drop(columns=['Data', 'ADV', 'Resultado', 'FG', 'FG%', '3PT', '3P%', 'FT', 'FT%'], inplace=True)

df_stats_jokic.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Placar,V_D,Casa_Fora,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,POR,107-99,D,fora,3,4,0.70,0.50,1.00
1,2016,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,UTAH,100-84,D,casa,6,4,0.78,0.00,0.83
2,2016,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,SA,102-98,V,casa,5,4,0.33,0.00,1.00
3,2016,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,OKC,124-102,D,casa,2,4,0.60,0.33,NaN
4,2016,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,SAC,115-106,D,casa,6,4,0.33,0.00,1.00


## Salva dataset ajustado

In [513]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset from csv

In [514]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
                             encoding='utf-8',
                             sep=',')
df_stats_jokic.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Placar,V_D,Casa_Fora,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,POR,107-99,D,fora,3,4,0.70,0.50,1.00
1,2016,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,UTAH,100-84,D,casa,6,4,0.78,0.00,0.83
2,2016,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,SA,102-98,V,casa,5,4,0.33,0.00,1.00
3,2016,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,OKC,124-102,D,casa,2,4,0.60,0.33,NaN
4,2016,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,SAC,115-106,D,casa,6,4,0.33,0.00,1.00


# Análise gráfica

## %stats

In [515]:
media_porcent_temporadas = df_stats_jokic[['Temporada', '%FG', '%FT', '%3PT']].groupby('Temporada').mean()

In [516]:
fig = px.line(media_porcent_temporadas,
              x=media_porcent_temporadas.index,
              y=['%FG', '%FT', '%3PT'],
              labels={'variable': ''})

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic %stats',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)
fig.update_yaxes(title_text=None)
fig.show()

## Pontos

In [517]:
media_pontos_temporadas = df_stats_jokic[['Temporada', 'Pontos']].groupby('Temporada').mean()

media_pontos_temporadas_por_tipo = df_stats_jokic[['Temporada', 'FG_Acertos', 'FT_Acertos', '3PT_Acertos']].groupby('Temporada').mean()

In [518]:
media_pontos_temporadas_por_tipo['FG_Pontos'] = media_pontos_temporadas_por_tipo['FG_Acertos'] * 2

media_pontos_temporadas_por_tipo['FT_Pontos'] = media_pontos_temporadas_por_tipo['FT_Acertos'] * 1

media_pontos_temporadas_por_tipo['3_Pontos'] = media_pontos_temporadas_por_tipo['3PT_Acertos'] * 3

media_pontos_temporadas_por_tipo

,FG_Acertos,FT_Acertos,3PT_Acertos,FG_Pontos,FT_Pontos,3_Pontos
Temporada,,,,,,
2016,3.837500,1.925000,0.350000,7.675000,1.925000,1.050000
2017,6.767123,2.575342,0.616438,13.534247,2.575342,1.849315
2018,6.720000,3.546667,1.480000,13.440000,3.546667,4.440000
2019,7.641975,3.567901,1.024691,15.283951,3.567901,3.074074
2020,7.662162,3.324324,1.094595,15.324324,3.324324,3.283784
2021,10.068493,4.684932,1.260274,20.136986,4.684932,3.780822
2022,10.240000,5.053333,1.320000,20.480000,5.053333,3.960000
2023,9.257143,4.871429,0.814286,18.514286,4.871429,2.442857


In [519]:
fig = make_subplots(shared_xaxes=True)

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['FG_Pontos'],
                         mode='lines',
                         line=dict(color='#845EC2'),
                         name='FG'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['FT_Pontos'],
                         mode='lines',
                         line=dict(color='#008BC9'),
                         name='FT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['3_Pontos'],
                         mode='lines',
                         line=dict(color='#FF8066'),
                         name='3PT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas.index,
                         y=media_pontos_temporadas['Pontos'],
                         mode='lines',
                         line=dict(color='#936C00'),
                         name='Média geral'))

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic - média de pontos nas temporadas',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)

fig.update_yaxes(title_text=None)

fig.show()

## Correlação

In [520]:
correlacao = df_stats_jokic[['Temporada',
                             'Minutos',
                             'Rebotes',
                             'Assistencias',
                             'Bloqueios',
                             'Desarmes',
                             'Faltas',
                             'Tunovers',
                             'Pontos',
                             'FG_Tentativas',
                             'FG_Acertos',
                             '3PT_Tentativas',
                             '3PT_Acertos',
                             'FT_Tentativas',
                             'FT_Acertos',
                             'Dia_semana',
                             'Mes',
                             '%FG',
                             '%3PT',
                             '%FT']].corr().round(2)

fig = px.imshow(correlacao, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1200,
                  height=1200)

fig.update_xaxes(side='top')

fig.show()

±0.9 → correlação muito forte

±0.7 a ±0.9 → correlação forte

±0.5 a ±0.7 → correlação moderada

±0.3 a ±0.5 → correlação fraca

### Correlação com pontos

#### Correlação muito forte

In [521]:
correlacao.loc[correlacao['Pontos'] >= 0.9].index

Index(['Pontos', 'FG_Acertos'], dtype='object')

#### Correlação forte

In [522]:
correlacao.loc[(correlacao['Pontos'] >= 0.7)
               & (correlacao['Pontos'] < 0.9)].index

Index(['FG_Tentativas'], dtype='object')

#### Correlação moderada

In [523]:
correlacao.loc[(correlacao['Pontos'] >= 0.5)
               & (correlacao['Pontos'] < 0.7)].index

Index(['Temporada', 'Minutos', 'FT_Tentativas', 'FT_Acertos'], dtype='object')

#### Correlação fraca

In [524]:
correlacao.loc[(correlacao['Pontos'] >= 0.3)
               & (correlacao['Pontos'] < 0.5)].index

Index(['Rebotes', 'Assistencias', '3PT_Tentativas', '3PT_Acertos', '%FG'], dtype='object')

### Conclusões

#### Pontuação

Os pontos do Jokic tem correlação:
* Muito forte com lances de quadra (FG)
* Moderada com lances livres (FT)
* Fraca com lances de 3 pontos (3PT)

In [525]:
media_acertos_fg = df_stats_jokic['FG_Acertos'].mean().round(2)
media_acertos_ft = df_stats_jokic['FT_Acertos'].mean().round(2)
media_acertos_3pt = df_stats_jokic['3PT_Acertos'].mean().round(2)

print('Média de acertos FG por jogo:', media_acertos_fg)
print('Média de acertos FT por jogo:', media_acertos_ft)
print('Média de acertos 3PT por jogo:', media_acertos_3pt)

Média de acertos FG por jogo: 7.72
Média de acertos FT por jogo: 3.67
Média de acertos 3PT por jogo: 0.99


#### Aproveitamento

In [526]:
media_aproveitamento_porcent_fg = (df_stats_jokic['%FG'].mean() * 100).round(2)
media_aproveitamento_porcent_ft = (df_stats_jokic['%FT'].mean() * 100).round(2)
media_aproveitamento_porcent_3pt = (df_stats_jokic['%3PT'].mean() * 100).round(2)

print('Média de aproveitamento FG por jogo:', media_aproveitamento_porcent_fg, '%')
print('Média de aproveitamento FT por jogo:', media_aproveitamento_porcent_ft, '%')
print('Média de aproveitamento 3PT por jogo:', media_aproveitamento_porcent_3pt, '%')

Média de aproveitamento FG por jogo: 55.78 %
Média de aproveitamento FT por jogo: 82.0 %
Média de aproveitamento 3PT por jogo: 35.06 %


A maior porcentagem de aproveitamento é de lances livres (FT)

Apesar do bom aproveitamento, a correlação é mais forte com FG por valer mais pontos que FT

#### Média de pontos

In [527]:
pontos_fg = 2
pontos_ft = 1
pontos_3pt = 3

media_pontos_fg = (df_stats_jokic['FG_Acertos'].mean() * pontos_fg).round(2)
media_pontos_ft = (df_stats_jokic['FT_Acertos'].mean() * pontos_ft).round(2)
media_pontos_3pt = (df_stats_jokic['3PT_Acertos'].mean() * pontos_3pt).round(2)
media_pontos_por_jogo = (df_stats_jokic['Pontos'].mean()).round(2)

print('Média de pontos FG:', media_pontos_fg)
print('Média de pontos FT:', media_pontos_ft)
print('Média de pontos 3PT:', media_pontos_3pt)
print('Média de pontos por jogo:', media_pontos_por_jogo)

Média de pontos FG: 15.45
Média de pontos FT: 3.67
Média de pontos 3PT: 2.98
Média de pontos por jogo: 20.11


# Substituição de variáveis categóricas

Objetivo
* Identificar quais componentes são essenciais para estimar o nº de pontos

In [528]:
df_stats_jokic_ohe = df_stats_jokic.copy()

## One Hot Encoder

In [529]:
nomes_valores_unicos = pd.concat([df_stats_jokic['Adversario'], df_stats_jokic['Casa_Fora']]).unique()

nomes_colunas = np.sort(np.array(nomes_valores_unicos))  # Ordem alfabética

nomes_colunas

array(['ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DET', 'DUR',
       'GIA', 'GS', 'HOU', 'IND', 'LAC', 'LAL', 'LEB', 'MEM', 'MIA',
       'MIL', 'MIN', 'NO', 'NY', 'OKC', 'ORL', 'PHI', 'PHX', 'POR', 'SA',
       'SAC', 'TOR', 'UTAH', 'WSH', 'casa', 'fora'], dtype=object)

In [530]:
ohe = OneHotEncoder()

adversario_casa_fora_sem_categoricas = ohe.fit_transform(df_stats_jokic[['Adversario', 'Casa_Fora']]).toarray()

In [531]:
df_stats_jokic_adv_casa_fora_ohe = pd.DataFrame(adversario_casa_fora_sem_categoricas, columns=nomes_colunas)

In [532]:
df_stats_jokic_adv_casa_fora_ohe

,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,DUR,GIA,GS,HOU,IND,LAC,LAL,LEB,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [533]:
vitoria_derrota_sem_repeticao = df_stats_jokic['V_D'].unique()

vitoria_derrota_colunas = np.sort(np.array(vitoria_derrota_sem_repeticao))  # Ordem alfabética

vitoria_derrota_colunas

array(['D', 'V'], dtype=object)

In [534]:
vitoria_derrota_sem_categoricas = ohe.fit_transform(df_stats_jokic[['V_D']]).toarray()

In [535]:
df_stats_jokic_vitoria_derrota_ohe = pd.DataFrame(vitoria_derrota_sem_categoricas, columns=vitoria_derrota_colunas)

In [536]:
df_stats_jokic_vitoria_derrota_ohe

,D,V
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0
...,...,...
596,0.0,1.0
597,1.0,0.0
598,0.0,1.0
599,0.0,1.0


In [537]:
df_stats_jokic_sem_categoricas = pd.concat([df_stats_jokic_adv_casa_fora_ohe, df_stats_jokic_vitoria_derrota_ohe], axis=1)

df_stats_jokic_sem_categoricas

,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,DUR,GIA,GS,HOU,IND,LAC,LAL,LEB,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora,D,V
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [538]:
df_stats_jokic_ohe = pd.concat([df_stats_jokic_ohe, df_stats_jokic_sem_categoricas], axis=1)

df_stats_jokic_ohe.drop(columns=['Adversario', 'Casa_Fora', 'V_D', 'Placar'], inplace=True)

In [539]:
df_stats_jokic_ohe.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Dia_semana,Mes,%FG,%3PT,%FT,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,DUR,GIA,GS,HOU,IND,LAC,LAL,LEB,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora,D,V
0,2016,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,3,4,0.70,0.50,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2016,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,6,4,0.78,0.00,0.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,2016,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,5,4,0.33,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,2016,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,2,4,0.60,0.33,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2016,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,6,4,0.33,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


## Correlação OHE

In [540]:
correlacao_ohe = df_stats_jokic_ohe.corr().round(2)

fig = px.imshow(correlacao_ohe, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1800,
                  height=1800)

fig.update_xaxes(side='top')

fig.show()

In [541]:
df_correlacao_ohe = pd.DataFrame(correlacao_ohe)

df_correlacao_ohe.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Dia_semana,Mes,%FG,%3PT,%FT,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,DUR,GIA,GS,HOU,IND,LAC,LAL,LEB,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,casa,fora,D,V
Temporada,1.00,0.46,0.34,0.55,0.01,0.13,-0.06,0.34,0.50,0.42,0.48,0.21,0.16,0.35,0.33,-0.05,-0.04,0.20,0.01,0.03,0.00,0.00,0.01,0.00,-0.01,0.00,-0.03,0.01,0.05,0.05,-0.02,0.01,-0.02,0.00,0.02,-0.01,0.01,0.00,-0.02,-0.01,0.01,-0.01,-0.00,0.01,0.03,-0.03,-0.01,-0.01,-0.00,0.00,-0.00,0.02,-0.01,0.01,-0.16,0.16
Minutos,0.46,1.00,0.57,0.52,0.20,0.22,0.12,0.38,0.68,0.69,0.65,0.45,0.34,0.43,0.42,0.01,-0.16,0.01,0.03,0.08,-0.02,-0.01,0.00,0.03,-0.06,0.02,-0.02,-0.00,-0.09,-0.12,-0.03,-0.09,0.02,-0.01,-0.00,-0.10,0.03,0.05,0.02,0.04,-0.02,-0.01,0.03,0.03,0.05,0.03,-0.02,0.05,0.02,-0.02,-0.00,0.00,0.01,-0.01,-0.12,0.12
Rebotes,0.34,0.57,1.00,0.35,0.13,0.14,0.05,0.28,0.44,0.47,0.43,0.19,0.11,0.32,0.30,0.02,-0.16,0.00,-0.02,0.02,0.02,0.04,-0.01,0.13,-0.02,0.01,0.04,-0.05,-0.03,-0.10,-0.01,-0.01,-0.04,0.02,-0.02,-0.01,-0.00,0.02,-0.04,0.00,-0.02,-0.01,0.00,-0.01,0.00,0.02,-0.05,-0.03,0.02,0.02,-0.02,0.03,0.09,-0.09,-0.23,0.23
Assistencias,0.55,0.52,0.35,1.00,0.08,0.10,0.02,0.28,0.33,0.24,0.31,0.10,0.13,0.24,0.23,0.02,-0.13,0.18,0.07,0.01,-0.01,-0.01,-0.01,0.02,0.04,0.02,-0.02,-0.01,-0.00,-0.05,-0.00,-0.01,-0.05,-0.03,-0.02,-0.08,-0.06,-0.02,0.08,0.01,0.02,-0.03,-0.00,0.08,0.05,-0.01,-0.02,0.06,0.02,-0.03,0.03,-0.03,0.16,-0.16,-0.30,0.30
Bloqueios,0.01,0.20,0.13,0.08,1.00,0.03,0.04,0.06,0.18,0.18,0.17,0.14,0.06,0.13,0.12,0.04,-0.04,-0.01,-0.01,-0.02,0.03,0.02,0.04,-0.02,0.04,-0.05,-0.04,0.03,-0.01,-0.05,-0.04,-0.00,-0.00,0.01,-0.01,-0.03,-0.04,-0.00,-0.01,0.02,0.02,0.01,0.04,-0.01,0.05,-0.01,0.01,-0.02,0.01,-0.00,-0.01,-0.02,0.02,-0.02,-0.02,0.02


In [544]:
df_correlacao_ohe.loc[(df_correlacao_ohe['Pontos'] > 0.5)
                      & (df_correlacao_ohe['Pontos'] != 1)].index

Index(['Minutos', 'FG_Tentativas', 'FG_Acertos', 'FT_Tentativas',
       'FT_Acertos'],
      dtype='object')